# AeroCom multi-model intercomparison of aerosol optical properties with observations

In [1]:
import os
import pyaerocom as pya
pya.change_verbosity('warning')

2018-08-07 15:55:13,665:INFO:
Reading aliases ini file: /home/jonasg/github/pyaerocom/pyaerocom/data/aliases.ini


## Define variables and years of interest

In [2]:
YEARS = [2010]
# Dictionary containing all variables to be read
 
VAR_INFO = dict(od550aer    = 'AOD at 550 nm',
                od550gt1aer = 'Coarse mode AOD at 550nm',
                od550lt1aer = 'Fine mode AOD at 550nm',
                abs550aer   = 'Absorption AOD at 550nm')
VARS = [var for var in VAR_INFO.keys()] 

### Obs networks
List of considered observation networks

In [3]:
obs_reader = pya.io.ReadUngridded()
print('Supported networks: {}'.format(obs_reader.supported_datasets))

Supported networks: ['AeronetInvV3Lev2.daily', 'AeronetInvV3Lev1.5.daily', 'AeronetInvV2Lev2.daily', 'AeronetInvV2Lev1.5.daily', 'AeronetSDAV2Lev2.daily', 'AeronetSDAV3Lev1.5.daily', 'AeronetSDAV3Lev2.daily', 'AeronetSunV2Lev2.daily', 'AeronetSunV2Lev2.AP', 'AeronetSunV3Lev1.5.daily', 'AeronetSunV3Lev1.5.AP', 'AeronetSunV3Lev2.daily', 'AeronetSunV3Lev2.AP', 'EARLINET', 'EBASMC']


In [4]:
from pyaerocom import const
# use only level 2 Aeronet for now and daily data
obs_networks =['AeronetSDAV2Lev2.daily', # od550aer, od550lt1aer, od550gt1aer
               'AeronetSDAV3Lev2.daily', # od550aer, od550lt1aer, od550gt1aer
               'AeronetSunV2Lev2.daily', # od550aer
               'AeronetSunV3Lev2.daily',
               ] # od550aer
               #'EBASMC'] # ec550aer, ...

### Load obs data

In [7]:
obs_data = {}
for network in obs_networks:
    reader = pya.io.ReadUngridded(network)
    read_vars = []
    for var_provided in reader.dataset_provides_variables(network):
        if var_provided in VARS:
            read_vars.append(var_provided)
    if len(read_vars) != 0:
        print('loading {}...'.format(network))
        obs_data[network] = reader.read(vars_to_retrieve=read_vars)
        print('Successfully loaded {} (variables: {})'.format(network, read_vars))

loading AeronetSDAV2Lev2.daily...
Successfully loaded AeronetSDAV2Lev2.daily (variables: ['od550aer', 'od550gt1aer', 'od550lt1aer'])
loading AeronetSDAV3Lev2.daily...
Successfully loaded AeronetSDAV3Lev2.daily (variables: ['od550aer', 'od550gt1aer', 'od550lt1aer'])
loading AeronetSunV2Lev2.daily...
Successfully loaded AeronetSunV2Lev2.daily (variables: ['od550aer'])
loading AeronetSunV3Lev2.daily...
Successfully loaded AeronetSunV3Lev2.daily (variables: ['od550aer'])


In [8]:
obs_data

{'AeronetSDAV2Lev2.daily': <pyaerocom.ungriddeddata.UngriddedData at 0x7f7d9467fb70>,
 'AeronetSDAV3Lev2.daily': <pyaerocom.ungriddeddata.UngriddedData at 0x7f7d9467ffd0>,
 'AeronetSunV2Lev2.daily': <pyaerocom.ungriddeddata.UngriddedData at 0x7f7d9419cd30>,
 'AeronetSunV3Lev2.daily': <pyaerocom.ungriddeddata.UngriddedData at 0x7f7d51960080>}

## Model-list

The following list comprises search strings for Aerocom models:

**NOTE**: Check and update from 

In [9]:
model_info_file = '/lustre/storeA/project/aerocom/aerocom-users-database/AEROCOM-PHASE-III/reference-list'

if os.path.exists(model_info_file):
    with open(model_info_file) as f:
        for line in f:
            if line.strip():
                print(line)

1 NorESM: AlfKirkevag MetNo

  CAM6-Oslo_NF2kNucl_7jun2018AK

attention

is on /lustre/storeA/project/aerocom/aerocom2/NorESM_SVN_TEST

2 OsloCTM3 : Gunnar Myhre - Marianne Lund CICERO

OsloCTM2_INSITU  (includes extinction and absorption coefficients)

is probably outdated...

3 TM5-AP3: Twan van Noye KNMI

TM5_AP3-CTRL2016

and

TM5_AP3-INSITU (daily files are missing, but Jan should produce them)



In [10]:
MODEL_SEARCH_KEYS = ['ECMWF_OSUITE',
                     'CAM6-Oslo_NF2kNucl_7jun2018AK',
                     'OsloCTM2_INSITU',
                     'TM5_AP3-CTRL2016',
                     'TM5_AP3-INSITU']

Search matches:

In [ ]:
from pyaerocom.io.aerocom_browser import AerocomBrowser
ALL_MODELS = []

browser = AerocomBrowser()

for search_key in MODEL_SEARCH_KEYS:
    try:
        ALL_MODELS.extend(browser.find_matches(search_key))
    except Exception as e:
        print(repr(e))
ALL_MODELS

### Check variables and years available in the listed models 

Keep only the models that match either of the specified years and either of the specified variables.

In [ ]:
#dictionary that will be filled with reading classes for each model
readers = {}
for model_id in ALL_MODELS:
    print('\n\nReading model {}'.format(model_id))
    reader = pya.io.ReadGridded(model_id)
    if any([year in reader.years for year in YEARS]): 
        ok = True
        if VARS is not None:
            if not any([var in reader.vars for var in VARS]):
                print('Ignoring model {}: no match with any of the '
                      'defined variables (contains:\n[])'.format(model_id, reader.vars))
                ok = False
    else:
        print('Ignoring model {}: no match with defined years (contains:\n{})'.format(model_id, reader.years))
        ok = False
    if ok:
        readers[model_id] = reader

In [ ]:
print('Remaining models:')
for k, v in readers.items():
    print('{}: {}'.format(k, v))

In [ ]:
ALL_VARS = []
for reader in readers.values():
    if len(reader.files) == 0:
        raise Exception
    ALL_VARS.extend(reader.vars)

ALL_VARS = list(dict.fromkeys(ALL_VARS))    
ALL_VARS

In [ ]:
readers.keys()

In [ ]:
reader = readers['CAM6-Oslo_NF2kNucl_7jun2018AK']
print(reader)